In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import models
from PIL import Image

In [ ]:
!pip install pytorch-pretrained-biggan



In [ ]:
import os
import shutil
import nltk

# Define the expected NLTK directory structure
nltk_data_path = '/kaggle/working/nltk_data/corpora/'
os.makedirs(nltk_data_path, exist_ok=True)

# Path to the uploaded wordnet directory
wordnet_source_path = '/kaggle/input/wordnetn/wordnet/'
wordnet_dest_path = os.path.join(nltk_data_path, 'wordnet')

# Move or copy the wordnet folder to the expected location
if not os.path.exists(wordnet_dest_path):
    shutil.copytree(wordnet_source_path, wordnet_dest_path)

# Add the working directory to NLTK's data path
nltk.data.path.append('/kaggle/working/nltk_data/')
# Test WordNet
from nltk.corpus import wordnet

try:
    syns = wordnet.synsets('example')
    print(f"WordNet loaded successfully. Example synsets: {syns}")
except Exception as e:
    print(f"Error loading WordNet: {e}")

In [ ]:
import nltk

nltk.data.path.append('/kaggle/working/nltk_data')
from nltk.corpus import wordnet

print(wordnet.synsets('horse'))

In [ ]:
import numpy as np
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import os
from PIL import Image
import torch
import torch.optim as optim
import torch.nn as nn
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample)
import random
import nltk
import timm
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



biggan = BigGAN.from_pretrained('biggan-deep-128').to('cuda')

classes_file = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/classes.txt"
predicate_matrix_file = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/predicate-matrix-continuous.txt"
train_dir = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/train"
def load_classes(file_path):
    with open(file_path, "r") as f:
        classes = [line.strip().split('\t')[-1] for line in f.readlines()]
    return classes

def load_and_normalize_predicate_matrix(file_path):
    predicate_matrix = np.loadtxt(file_path)
    predicate_matrix = (predicate_matrix - predicate_matrix.min(axis=0)) / (predicate_matrix.max(axis=0) - predicate_matrix.min(axis=0) + 1e-8)
    return predicate_matrix

classes = load_classes(classes_file)
predicate_matrix = load_and_normalize_predicate_matrix(predicate_matrix_file)

folder_classes = set(os.listdir(train_dir))
unseen_classes = [cls for cls in classes if cls not in folder_classes]
print(f"Unseen classes: {unseen_classes}")

class ImageDataset(Dataset):
    def __init__(self, root_dir, class_to_idx, transform=None):
        self.root_dir = root_dir
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.data = []
        for class_name in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                class_idx = class_to_idx[class_name]
                for img_name in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_name)
                    self.data.append((img_path, class_idx))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),  # Randomly flip the image vertically
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomRotation(30),  # Increased rotation angle
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random affine transformations
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
dataset = ImageDataset(train_dir, class_to_idx, transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8)

unseen_class_to_idx = {cls: idx + len(folder_classes) for idx, cls in enumerate(unseen_classes)}
# Manual synset mapping for problematic classes
manual_synsets = {
    "moose": "elk.n.01",
    "rhinoceros": "rhinoceros.n.01",
    "squirrel": "squirrel.n.01"
}

generated_data = []
for class_name in unseen_classes:
    try:
        print(f"Generating images for class: {class_name}")
        synsets = wordnet.synsets(class_name.replace("+", " "))
        # Use manual mapping if the synset is not directly available
        synset_name = manual_synsets.get(class_name) if class_name in manual_synsets else synsets[0].name()
        if not synset_name:
            print(f"No valid synset found for {class_name}, skipping.")
            continue

        class_idx = unseen_class_to_idx[class_name]
        first_image_logged = False
        for batch_start in range(0, 250, 50):
            batch_size = min(50, 250 - batch_start)
            class_vector = torch.tensor(
                one_hot_from_names([synset_name.split('.')[0]], batch_size=batch_size),
                dtype=torch.float32
            ).to('cuda')
            noise_vector = torch.tensor(
                truncated_noise_sample(truncation=0.4, batch_size=batch_size, seed=42),
                dtype=torch.float32
            ).to('cuda')

            with torch.no_grad():
                output = biggan(noise_vector, class_vector, truncation=0.4)
            for i in range(output.shape[0]):
                image = transforms.ToPILImage()(output[i].cpu().squeeze())
                generated_data.append((image, class_idx))
                if not first_image_logged:
                    plt.imshow(image)
                    plt.title(f"Generated Image for {class_name}")
                    plt.axis('off')
                    plt.show()
                    first_image_logged = True
            print(f"Generated {len(generated_data)} images for {class_name}")
    except Exception as e:
        print(f"Error generating image for {class_name}: {e}")




# Add generated data to the dataset

for img, label in generated_data:
    dataset.data.append((img, label))

print(f"Original dataset size: {len(train_dataset) + len(val_dataset)}")
print(f"Generated dataset size: {len(generated_data)}")
print(f"Total dataset size after augmentation: {len(dataset)}")
# Model initialization using Swin Transformer
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=50)
model = model.to(device)

for param in model.parameters():
    param.requires_grad = True


# Define loss, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-3) 
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.005, total_steps=9 * len(train_loader))


# Training and validation loop
num_epochs = 9
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metrics calculation
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Train Loss: {loss.item():.4f}, Train Acc: {100 * correct_train / total_train:.2f}%")

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Metrics calculation
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    val_loss /= len(val_loader)
    val_acc = 100 * correct_val / total_val

    # Scheduler step
    scheduler.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}] Complete, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

print("Training complete.")


In [ ]:
import os
import pandas as pd
from PIL import Image
import torch


# Prediction
test_dir = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/test"
model.eval()
test_images = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]
test_predictions = []

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

for img_name in test_images:
    img_path = os.path.join(test_dir, img_name)
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        predicted_class = torch.argmax(outputs, dim=1).item()
        class_name = classes[predicted_class]  # Get the class name
        test_predictions.append((img_name, class_name))

# Save Predictions
submission = pd.DataFrame(test_predictions, columns=['image_id', 'class'])
submission.to_csv("/kaggle/working/submission.csv", index=False)